In [37]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [38]:
from typing import List, Dict, Any, Union

In [39]:
!pip install ibm-db==3.1.2

In [40]:
import base64
import ibm_db
import ibm_db_dbi
from kubernetes import client, config, utils
import yaml
from http import HTTPStatus

config.load_incluster_config()

with open("/var/run/secrets/kubernetes.io/serviceaccount/namespace", "r") as f:
    NAMESPACE = f.read()

core_v1_api = client.CoreV1Api()

# Gather dataset column names and levels into configmap

In [41]:
import pandas as pd
import numpy as np
import json

df = pd.read_csv("./german_credit_data_biased_training.csv")

for col in df.columns:
    if df[col].dtype == np.dtype("O"):
        df[col] = df[col].astype("category")

column_map: Dict[str, Any] = {}
column_map["columns"] = list(df.columns)

column_map["label_columns"] = {
    col: list(df[col].dtype.categories)
    for col in column_map["columns"]
    if type(df[col].dtype) == pd.core.dtypes.dtypes.CategoricalDtype
}
column_map["int_columns"] = [
    col for col in column_map["columns"] if df[col].dtype == np.dtype("int64")
]

column_map

{'columns': ['CheckingStatus',
  'LoanDuration',
  'CreditHistory',
  'LoanPurpose',
  'LoanAmount',
  'ExistingSavings',
  'EmploymentDuration',
  'InstallmentPercent',
  'Sex',
  'OthersOnLoan',
  'CurrentResidenceDuration',
  'OwnsProperty',
  'Age',
  'InstallmentPlans',
  'Housing',
  'ExistingCreditsCount',
  'Job',
  'Dependents',
  'Telephone',
  'ForeignWorker',
  'Risk'],
 'label_columns': {'CheckingStatus': ['0_to_200',
   'greater_200',
   'less_0',
   'no_checking'],
  'CreditHistory': ['all_credits_paid_back',
   'credits_paid_to_date',
   'no_credits',
   'outstanding_credit',
   'prior_payments_delayed'],
  'LoanPurpose': ['appliances',
   'business',
   'car_new',
   'car_used',
   'education',
   'furniture',
   'other',
   'radio_tv',
   'repairs',
   'retraining',
   'vacation'],
  'ExistingSavings': ['100_to_500',
   '500_to_1000',
   'greater_1000',
   'less_100',
   'unknown'],
  'EmploymentDuration': ['1_to_4',
   '4_to_7',
   'greater_7',
   'less_1',
   'unemp

In [42]:
COLUMNS_CONFIG_MAP_NAME = "credit-risk-columns"
cm = client.V1ConfigMap(
    metadata=client.V1ObjectMeta(name=COLUMNS_CONFIG_MAP_NAME, namespace=NAMESPACE),
    data={"columns": json.dumps(column_map, indent=2)},
)

try:
    core_v1_api.create_namespaced_config_map(namespace=NAMESPACE, body=cm)
except client.ApiException as e:
    if e.status == HTTPStatus.CONFLICT:
        core_v1_api.patch_namespaced_config_map(
            namespace=NAMESPACE, name=COLUMNS_CONFIG_MAP_NAME, body=cm
        )
    else:
        raise

# Split data into test and train datasets

In [43]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, train_size=0.8, random_state=42, shuffle=True)

# DB2

## Create Secret with DB2 Credentials

In [44]:
with open("db2-secret.yaml", "r") as f:
    desc = yaml.safe_load(f)
    try:
        api.create_namespaced_secret(body=desc, namespace=NAMESPACE)
    except client.ApiException as e:
        if e.status == HTTPStatus.CONFLICT:
            api.patch_namespaced_secret(
                body=desc, name=desc["metadata"]["name"], namespace=NAMESPACE
            )
        else:
            raise e

In [45]:
def get_db2_conn(core_v1_api: client.CoreV1Api) -> ibm_db.IBM_DBConnection:
    secret = core_v1_api.read_namespaced_secret("db2-credentials", NAMESPACE)

    to_str = lambda b64_data: base64.b64decode(b64_data).decode("utf-8")
    host, username, password, port = (
        to_str(secret.data["host"]),
        to_str(secret.data["username"]),
        to_str(secret.data["password"]),
        to_str(secret.data["port"]),
    )

    conn_str = (
        "DRIVER={IBM DB2 ODBC DRIVER};"
        f"DATABASE=BLUDB;HOSTNAME={host};PORT={port};PROTOCOL=TCPIP;UID={username};Pwd={password};SECURITY=SSL;"
    )

    conn = ibm_db.connect(conn_str, "", "")

    return conn

## Verify DB2 Connection

In [46]:
import logging
import textwrap

log = logging.getLogger("SETUP")

try:
    db2 = get_db2_conn(api)
except NameError:
    raise
except Exception as e:  # no better exception to catch
    log.error(e)
    log.error(
        textwrap.dedent(
            """
           DB2 connection failed! Please check that db2-secret.yaml in this directory contains the correct connection details. 
           Re-run the script to recreate the secret
        """
        )
    )
    exit()

## DB2 Utils

In [55]:
def table_exists(table_name: str, conn: ibm_db.IBM_DBConnection) -> Union[dict, bool]:
    stmt = ibm_db.prepare(
        conn,
        "SELECT * FROM SYSIBM.TABLES WHERE table_name = ? AND "
        "table_schema = CURRENT_SCHEMA",
    )
    _ = ibm_db.execute(stmt, (table_name,))
    return ibm_db.fetch_assoc(stmt)


def build_check_constraints_sql(
    table_name: str, column_info: Dict[str, Any]
) -> List[str]:
    constraints = []
    for label_col, levels in column_info["label_columns"].items():
        safe_col_name = '"' + label_col.replace('"', "") + '"'
        str_levels = ["'" + level.replace("'", "''") + "'" for level in levels]
        constraints.append(f"CHECK ({safe_col_name} IN ({','.join(str_levels)}))")
    return constraints


def build_create_table_sql(
    name: str, column_info: Dict[str, Any], identity_start: int = 1
) -> str:
    sql_safe_table_name = '"' + name.replace('"', "") + '"'

    cols_sql = []
    for col in column_info["label_columns"]:
        safe_col_name = '"' + col.replace('"', "") + '"'
        cols_sql.append(f"{safe_col_name} VARCHAR(32000) NOT NULL")
    for col in column_info["int_columns"]:
        safe_col_name = '"' + col.replace('"', "") + '"'
        cols_sql.append(f"{safe_col_name} BIGINT NOT NULL")

    return (
        f"CREATE TABLE {sql_safe_table_name} ("
        + f"CLIENT_ID BIGINT GENERATED BY DEFAULT AS IDENTITY (START WITH {identity_start}),"
        + ", ".join(cols_sql)
        + ", "
        + ", ".join(build_check_constraints_sql(name, column_info))
        + ")"
    )


def df_to_sql(
    name: str,
    conn: ibm_db.IBM_DBConnection,
    df: pd.DataFrame,
    columns: Dict[str, Any],
    identity_start: int = 1,
) -> None:
    sql_safe_name = name.replace('"', "")
    if table_exists(name, conn):
        ibm_db.exec_immediate(conn, f'DROP TABLE "{sql_safe_name}"')

    sql = build_create_table_sql(sql_safe_name, columns, identity_start)
    ibm_db.exec_immediate(conn, sql)

    iStmtColsSql = ",".join([f'"{col}"' for col in columns["columns"]])
    iValues = ",".join(["?" for _ in range(len(columns["columns"]))])
    iSql = f'INSERT INTO "{sql_safe_name}" ({iStmtColsSql}) VALUES({iValues})'

    stmt = ibm_db.prepare(conn, iSql)
    df_values = tuple([tuple(x) for x in df.loc[:, columns["columns"]].values])
    ibm_db.execute_many(stmt, df_values)

### Create Test and Training tables

In [56]:
df_to_sql("TRAIN", db2, train, column_map)
df_to_sql("TEST", db2, test, column_map, identity_start=len(train) + 1)